In [11]:
import tensorflow as tf
import math
import numpy as np
import argparse
import os
import sys
import collections
import zipfile
import random
from pathlib import Path

In [2]:
# Give a folder path as an argument with '--log_dir' to save
# TensorBoard summaries. Default is a log folder in current directory.
current_path = os.path.dirname(os.path.realpath(sys.argv[0]))

parser = argparse.ArgumentParser()
parser.add_argument(
    '--log_dir',
    type=str,
    default=os.path.join(current_path, 'log'),
    help='The log directory for TensorBoard summaries.')
FLAGS, unparsed = parser.parse_known_args()

#### Read data

In [ ]:
vocab_size = 50000
batch_size = 128
embed_size = 128  # Dimension of the embedding vector.
skip_window = 1  # How many words to consider left and right.
num_skips = 2  # How many times to reuse an input to generate a label.
num_sampled = 64  # Number of negative examples to sample.
l_rate = 1.0

In [8]:
# def read_data(filename):
#   """Extract the first file enclosed in a zip file as a list of words."""
#   with zipfile.ZipFile(filename) as f:
#         data = tf.compat.as_str(f.read(f.namelist()[0])).split()
#   return data

In [48]:
def read_data(fname):
    print(fname)
    rws = np.concatenate([
        np.loadtxt(f.open(), delimiter='\t', dtype=int)
        for f in rw_location.glob(fname)
        if f.stat().st_size > 0
    ])
    return rws

In [66]:
rw_location = Path("/Users/Ganymedian/Desktop/dynamic-rw/output/karate-test/")
filename = "test.txt"
data = read_data(filename)
print('Data size', len(vocabulary))
print(vocabulary)

test.txt
Data size 6
[[1 2]
 [2 1]
 [2 3]
 [3 2]
 [3 4]
 [4 3]]


In [26]:
# # Step 2: Build the dictionary and replace rare words with UNK token.
# def build_dataset(words, n_words):
#   """Process raw inputs into a dataset."""
#   count = [['UNK', -1]]
#   count.extend(collections.Counter(words[0]).most_common(n_words - 1))
#   dictionary = dict()
#   for word, _ in count:
#     dictionary[word] = len(dictionary) # assigns incremental ID to each word starting from zero.
#   data = list()
#   unk_count = 0
#   for word in words[0]:
#     index = dictionary.get(word, 0)
#     if index == 0:  # dictionary['UNK']
#       unk_count += 1
#     data.append(index)
#   count[0][1] = unk_count
#   reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
#   return data, count, dictionary, reversed_dictionary

In [27]:
# vocab_size= 100
# data, count, dictionary, reverse_dictionary = build_dataset(vocabulary, vocab_size)
# print(data)

[1, 2, 3, 4]


In [30]:
# data_index = 0
# def generate_batch(batch_size, num_skips, window):
#   global data_index
#   assert batch_size % num_skips == 0
#   assert num_skips <= 2 * window
#   batch = np.ndarray(shape=(batch_size), dtype=np.int32)
#   contexts = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
#   span = 2 * window + 1  # [ window target window ]
#   buffer = collections.deque(maxlen=span)  # pylint: disable=redefined-builtin
#   if data_index + span > len(data):
#     data_index = 0
#   buffer.extend(data[data_index:data_index + span])
#   data_index += span
#   for i in range(batch_size // num_skips):
#     context_words = [w for w in range(span) if w != window]
#     words_to_use = random.sample(context_words, num_skips) # without replacement
#     for j, context_word in enumerate(words_to_use):
#       batch[i * num_skips + j] = buffer[window]
#       contexts[i * num_skips + j, 0] = buffer[context_word]
#     if data_index == len(data):
#       buffer.extend(data[0:span])
#       data_index = span
#     else:
#       buffer.append(data[data_index])
#       data_index += 1
#   # Backtrack a little bit to avoid skipping words in the end of a batch
#   data_index = (data_index + len(data) - span) % len(data)
#   return batch, contexts

In [88]:
b = np.vsplit(data, 2)[0]
print(b)
print(b[:,1].reshape(len(b),1))
batch = np.ndarray(shape=(2, 1), dtype=np.int32)
print(batch)

[[1 2]
 [2 1]
 [2 3]]
[[2]
 [1]
 [3]]
[[1]
 [0]]


In [104]:

def generate_batch():
    global data_index
    next_batch = batches[data_index]
    data_index += 1
    targets = next_batch[:,0]
    contexts = next_batch[:,1].reshape(len(next_batch),1)
    return targets, contexts

In [110]:
data_index = 0
batch_size = 2
data_size = len(data)
batches = np.vsplit(data, data_size/batch_size)
print(len(batches))
batch, labels = generate_batch()
for i in range(batch_size):
    print(batch[i], '->', labels[i, 0])

3
1 -> 2
2 -> 1


In [38]:
# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. These 3 variables are used only for
# displaying model accuracy, they don't affect calculation.
valid_size = 16  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [ ]:
for inputs, contexts in generate_batch(...):
  feed_dict = {train_inputs: inputs, train_contexts: contexts}
  _, cur_loss = session.run([optimizer, loss], feed_dict=feed_dict)

In [ ]:
graph = tf.Graph()

with graph.as_default():

  # Input data.
  with tf.name_scope('inputs'):
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_contexts = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    with tf.name_scope('embeddings'):
        embeddings = tf.Variable(
              tf.random_uniform([vocab_size, embed_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    with tf.name_scope('weights'):
          nce_weights = tf.Variable(
              tf.truncated_normal(
                  [vocab_size, embed_size],
                  stddev=1.0 / math.sqrt(embed_size)))
    with tf.name_scope('biases'):
          nce_biases = tf.Variable(tf.zeros([vocab_size]))
  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  # Explanation of the meaning of NCE loss:
  #   http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(
            tf.nn.nce_loss(
                weights=nce_weights,
                biases=nce_biases,
                labels=train_contexts,
                inputs=embed,
                num_sampled=num_sampled,
                num_classes=vocab_size))
    # Add the loss value as a scalar to summary.
    tf.summary.scalar('loss', loss)
    # Construct the SGD optimizer using a learning rate of 1.0.
    with tf.name_scope('optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(l_rate).minimize(loss)
    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings,
                                            valid_dataset)
    similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

    # Merge all summaries.
    merged = tf.summary.merge_all()

    # Add variable initializer.
    init = tf.global_variables_initializer()

    # Create a saver.
    saver = tf.train.Saver()

### Begin Training

In [ ]:
num_steps = 100001

with tf.Session(graph=graph) as session:
    # Open a writer to write summaries.
    writer = tf.summary.FileWriter(FLAGS.log_dir, session.graph)

    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips,
                                                    skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # Define metadata variable.
        run_metadata = tf.RunMetadata()

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        # Also, evaluate the merged op to get all summaries from the returned "summary" variable.
        # Feed metadata variable to session for visualizing the graph in TensorBoard.
        _, summary, loss_val = session.run(
            [optimizer, merged, loss],
            feed_dict=feed_dict,
            run_metadata=run_metadata)
        average_loss += loss_val

        # Add returned summaries to writer in each step.
        writer.add_summary(summary, step)
        # Add metadata to visualize the graph for the last run.
        if step == (num_steps - 1):
          writer.add_run_metadata(run_metadata, 'step%d' % step)

        if step % 2000 == 0:
          if step > 0:
            average_loss /= 2000
          # The average loss is an estimate of the loss over the last 2000 batches.
          print('Average loss at step ', step, ': ', average_loss)
          average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
          sim = similarity.eval()
          for i in xrange(valid_size):
            valid_word = reverse_dictionary[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            nearest = (-sim[i, :]).argsort()[1:top_k + 1]
            log_str = 'Nearest to %s:' % valid_word
            for k in xrange(top_k):
              close_word = reverse_dictionary[nearest[k]]
              log_str = '%s %s,' % (log_str, close_word)
            print(log_str)

    final_embeddings = normalized_embeddings.eval()
    # Write corresponding labels for the embeddings.

    with open(FLAGS.log_dir + '/metadata.tsv', 'w') as f:
        for i in xrange(vocab_size):
          f.write(reverse_dictionary[i] + '\n')

    # Save the model for checkpoints.
    saver.save(session, os.path.join(FLAGS.log_dir, 'model.ckpt'))

    # Create a configuration for visualizing embeddings with the labels in TensorBoard.
    config = projector.ProjectorConfig()
    embedding_conf = config.embeddings.add()
    embedding_conf.tensor_name = embeddings.name
    embedding_conf.metadata_path = os.path.join(FLAGS.log_dir, 'metadata.tsv')
    projector.visualize_embeddings(writer, config)

    writer.close()

### Step 6: Visualize the embeddings.

In [ ]:
# pylint: disable=missing-docstring
# Function to draw visualization of distance between embeddings.
def plot_with_labels(low_dim_embs, labels, filename):
  assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(
        label,
        xy=(x, y),
        xytext=(5, 2),
        textcoords='offset points',
        ha='right',
        va='bottom')

  plt.savefig(filename)

try:
  # pylint: disable=g-import-not-at-top
  from sklearn.manifold import TSNE
  import matplotlib.pyplot as plt

  tsne = TSNE(
      perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
  plot_only = 500
  low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
  labels = [reverse_dictionary[i] for i in xrange(plot_only)]
  plot_with_labels(low_dim_embs, labels, os.path.join(gettempdir(), 'tsne.png'))

except ImportError as ex:
  print('Please install sklearn, matplotlib, and scipy to show embeddings.')
  print(ex)